
step 1: 
Read files from Bronze and enforce schema and save as delta tables in silver layer, do some sanity checks, pattiton the file

Read schema from json files

Step 2: 
Ready Silver layer tables and to agg's and store data in gold layer for reading.




In [0]:
%fs ls /FileStore/shared_uploads/bkdevde@yahoo.com/raw/

path,name,size,modificationTime
dbfs:/FileStore/shared_uploads/bkdevde@yahoo.com/raw/Ball_By_Ball.csv,Ball_By_Ball.csv,24816834,1718995658000
dbfs:/FileStore/shared_uploads/bkdevde@yahoo.com/raw/Match.csv,Match.csv,114194,1718998673000
dbfs:/FileStore/shared_uploads/bkdevde@yahoo.com/raw/Player.csv,Player.csv,35160,1718995940000
dbfs:/FileStore/shared_uploads/bkdevde@yahoo.com/raw/Player_match.csv,Player_match.csv,2633931,1727974988000
dbfs:/FileStore/shared_uploads/bkdevde@yahoo.com/raw/Team.csv,Team.csv,341,1718995653000


In [0]:
dbutils.fs.rm('dbfs:/FileStore/shared_uploads/bkdevde@yahoo.com/raw/Player_match.csv')

Out[1]: True

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType,DecimalType
import json


# Initialize SparkSession
spark = SparkSession.builder.appName("IPL_Orange_Cap_Stats").getOrCreate()


#schema's
ball_by_ball_stats_schema = StructType([
    StructField("Match_id", IntegerType(), False),
    StructField("Over_id", IntegerType(), True),
    StructField("Ball_id", IntegerType(), True),
    StructField("Innings_No", IntegerType(), True),
    StructField("Team_Batting", IntegerType(), True),
    StructField("Team_Bowling", IntegerType(), True),
    StructField("Striker_Batting_Position", IntegerType(), True),
    StructField("Extra_Type", StringType(), True),
    StructField("Runs_Scored", IntegerType(), True),
    StructField("Extra_runs", IntegerType(), True),
    StructField("Wides", IntegerType(), True),
    StructField("Legbyes", IntegerType(), True),
    StructField("Byes", IntegerType(), True),
    StructField("Noballs", IntegerType(), True),
    StructField("Penalty", IntegerType(), True),
    StructField("Bowler_Extras", IntegerType(), True),
    StructField("Out_type", StringType(), True),
    StructField("Caught", IntegerType(), True),
    StructField("Bowled", IntegerType(), True),
    StructField("Run_out", IntegerType(), True),
    StructField("LBW", IntegerType(), True),
    StructField("Retired_hurt", IntegerType(), True),
    StructField("Stumped", IntegerType(), True),
    StructField("caught_and_bowled", IntegerType(), True),
    StructField("hit_wicket", IntegerType(), True),
    StructField("ObstructingFeild", IntegerType(), True),
    StructField("Bowler_Wicket", IntegerType(), True),
    StructField("Match_Date", DateType(), True),
    StructField("Season", IntegerType(), True),
    StructField("Striker", IntegerType(), True),
    StructField("Non_Striker", IntegerType(), True),
    StructField("Bowler", IntegerType(), True),
    StructField("Player_Out", IntegerType(), True),
    StructField("Fielders", IntegerType(), True),
    StructField("Striker_match_SK", IntegerType(), True),
    StructField("StrikerSK", IntegerType(), True),
    StructField("NonStriker_match_SK", IntegerType(), True),
    StructField("NONStriker_SK", IntegerType(), True),
    StructField("Fielder_match_SK", IntegerType(), True),
    StructField("Fielder_SK", IntegerType(), True),
    StructField("Bowler_match_SK", IntegerType(), True),
    StructField("BOWLER_SK", IntegerType(), True),
    StructField("PlayerOut_match_SK", IntegerType(), True),
    StructField("BattingTeam_SK", IntegerType(), True),
    StructField("BowlingTeam_SK", IntegerType(), True),
    StructField("Keeper_Catch", IntegerType(), True),
    StructField("Player_out_sk", IntegerType(), True),
    StructField("MatchDateSK", IntegerType(), True)  # Changed type from DATA to IntegerType
])

player_match_schema= StructType([
    StructField("Player_match_SK", IntegerType(), False),
    StructField("PlayerMatch_key", DecimalType(20, 0), False),  # DECIMAL(20)
    StructField("Match_Id", IntegerType(), False),
    StructField("Player_Id", IntegerType(), False),
    StructField("Player_Name", StringType(), True),  # VARCHAR(255)
    StructField("DOB", DateType(), True),  # DATE
    StructField("Batting_hand", StringType(), True),  # VARCHAR(50)
    StructField("Bowling_skill", StringType(), True),  # VARCHAR(50)
    StructField("Country_Name", StringType(), True),  # VARCHAR(100)
    StructField("Role_Desc", StringType(), True),  # VARCHAR(100)
    StructField("Player_team", StringType(), True),  # VARCHAR(100)
    StructField("Opposit_Team", StringType(), True),  # VARCHAR(100)
    StructField("Season_year", IntegerType(), True),
    StructField("is_manofThematch", IntegerType(), True),
    StructField("Age_As_on_match", IntegerType(), True),
    StructField("IsPlayers_Team_won", IntegerType(), True),
    StructField("Batting_Status", StringType(), True),  # VARCHAR(50)
    StructField("Bowling_Status", StringType(), True),  # VARCHAR(50)
    StructField("Player_Captain", StringType(), True),  # VARCHAR(50)
    StructField("Opposit_captain", StringType(), True),  # VARCHAR(50)
    StructField("Player_keeper", StringType(), True),  # VARCHAR(50)
    StructField("Opposit_keeper", StringType(), True)  # VARCHAR(50)
])

players_data_schema = StructType([
    StructField("PLAYER_SK", IntegerType(), nullable=True),
    StructField("Player_Id", IntegerType(), nullable=True),
    StructField("Player_Name", StringType(), nullable=True),
    StructField("DOB", DateType(), nullable=True),
    StructField("Batting_hand", StringType(), nullable=True),
    StructField("Bowling_skill", StringType(), nullable=True),
    StructField("Country_Name", StringType(), nullable=True)
])


match_schema = StructType([
    StructField("Match_SK", IntegerType(), nullable=False),
    StructField("match_id", IntegerType(), nullable=False),
    StructField("Team1", StringType(), nullable=True),
    StructField("Team2", StringType(), nullable=True),
    StructField("match_date", DateType(), nullable=True),
    StructField("Season_Year", IntegerType(), nullable=True),
    StructField("Venue_Name", StringType(), nullable=True),
    StructField("City_Name", StringType(), nullable=True),
    StructField("Country_Name", StringType(), nullable=True),
    StructField("Toss_Winner", StringType(), nullable=True),
    StructField("match_winner", StringType(), nullable=True),
    StructField("Toss_Name", StringType(), nullable=True),
    StructField("Win_Type", StringType(), nullable=True),
    StructField("Outcome_Type", StringType(), nullable=True),
    StructField("ManOfMach", StringType(), nullable=True),
    StructField("Win_Margin", IntegerType(), nullable=True),
    StructField("Country_id", IntegerType(), nullable=True)
])

teams_schema = StructType([
    StructField("team_sk", IntegerType(), nullable=False),
    StructField("team_id", IntegerType(), nullable=False),
    StructField("team_name", StringType(), nullable=True)])



In [0]:
# Path to the CSV file's
dbfs_raw_file_path = 'dbfs:/FileStore/shared_uploads/bkdevde@yahoo.com/raw/'

from pyspark.sql.functions import split, col, expr, concat_ws
# Read the CSV file's into a DataFrame



ball_by_ball_stats_df = spark.read.csv(dbfs_raw_file_path +'Ball_By_Ball.csv', header=True, schema=ball_by_ball_stats_schema,mode="FAILFAST")

player_match_df = spark.read.csv(dbfs_raw_file_path +'Player_match.csv', header=True, schema=player_match_schema,mode="FAILFAST")

players_data_df= spark.read.csv(dbfs_raw_file_path +'Player.csv', header=True, schema=players_data_schema,mode="FAILFAST")

match_df= spark.read.csv(dbfs_raw_file_path +'Match.csv', header=True, schema=match_schema,mode="PERMISSIVE")

teams_df= spark.read.csv(dbfs_raw_file_path +'Team.csv', header=True, schema=teams_schema,mode="FAILFAST")


#--------------------------------------  Partition and Write as Delta tables in delta layer ------------------- 
ball_by_ball_stats_df.write.format("delta").mode("overwrite").option("path", "/FileStore/shared_uploads/bkdevde@yahoo.com/delta/ball_by_ball_stats").partitionBy('season').saveAsTable("ball_by_ball_stats")

players_data_df.write.format("delta").mode("overwrite").option("path", "/FileStore/shared_uploads/bkdevde@yahoo.com/delta/players_data").saveAsTable("players_data")

match_df.write.format("delta").mode("overwrite").option("path", "/FileStore/shared_uploads/bkdevde@yahoo.com/delta/match").saveAsTable("matches")


## Add new fileds
teams_df=teams_df.withColumn("team_name_abbrivation",
                             expr("concat_ws('', transform(split(team_name, ' '),x->substring(x,1,1)))"))

player_match_df=player_match_df.withColumn("player_team_abbrivation",
                             expr("concat_ws('', transform(split(player_team, ' '),x->substring(x,1,1)))")).withColumn("Opposit_Team_abbrivation",expr("concat_ws('', transform(split(Opposit_Team, ' '),x->substring(x,1,1)))"))



player_match_df.write.format("delta").mode("overwrite").option("path", "/FileStore/shared_uploads/bkdevde@yahoo.com/delta/player_match").option("mergeSchema", "true").partitionBy("season_year").saveAsTable("player_match")


teams_df.write.format("delta").mode("overwrite").option("path", "/FileStore/shared_uploads/bkdevde@yahoo.com/delta/teams").option("mergeSchema", "true").saveAsTable("teams")



+---------------+---------------+--------+---------+---------------+----------+--------------+--------------------+------------+---------+--------------------+--------------------+-----------+----------------+---------------+------------------+--------------+--------------+--------------+---------------+-------------+--------------+-----------------------+------------------------+
|Player_match_SK|PlayerMatch_key|Match_Id|Player_Id|    Player_Name|       DOB|  Batting_hand|       Bowling_skill|Country_Name|Role_Desc|         Player_team|        Opposit_Team|Season_year|is_manofThematch|Age_As_on_match|IsPlayers_Team_won|Batting_Status|Bowling_Status|Player_Captain|Opposit_captain|Player_keeper|Opposit_keeper|player_team_abbrivation|Opposit_Team_abbrivation|
+---------------+---------------+--------+---------+---------------+----------+--------------+--------------------+------------+---------+--------------------+--------------------+-----------+----------------+---------------+-------

In [0]:
match_df.printSchema()

root
 |-- Match_SK: integer (nullable = true)
 |-- match_id: integer (nullable = true)
 |-- Team1: string (nullable = true)
 |-- Team2: string (nullable = true)
 |-- match_date: date (nullable = true)
 |-- Season_Year: integer (nullable = true)
 |-- Venue_Name: string (nullable = true)
 |-- City_Name: string (nullable = true)
 |-- Country_Name: string (nullable = true)
 |-- Toss_Winner: string (nullable = true)
 |-- match_winner: string (nullable = true)
 |-- Toss_Name: string (nullable = true)
 |-- Win_Type: string (nullable = true)
 |-- Outcome_Type: string (nullable = true)
 |-- ManOfMach: string (nullable = true)
 |-- Win_Margin: integer (nullable = true)
 |-- Country_id: integer (nullable = true)



In [0]:
%sql
select COUNT(*) AS record_count FROM delta.`/FileStore/shared_uploads/bkdevde@yahoo.com/delta/player_match` where season_year=2009

record_count
1254


In [0]:
%fs ls /FileStore/shared_uploads/bkdevde@yahoo.com/delta

path,name,size,modificationTime
dbfs:/FileStore/shared_uploads/bkdevde@yahoo.com/delta/ball_by_ball_stats/,ball_by_ball_stats/,0,0
dbfs:/FileStore/shared_uploads/bkdevde@yahoo.com/delta/gold/,gold/,0,0
dbfs:/FileStore/shared_uploads/bkdevde@yahoo.com/delta/match/,match/,0,0
dbfs:/FileStore/shared_uploads/bkdevde@yahoo.com/delta/player_match/,player_match/,0,0
dbfs:/FileStore/shared_uploads/bkdevde@yahoo.com/delta/players_data/,players_data/,0,0
dbfs:/FileStore/shared_uploads/bkdevde@yahoo.com/delta/teams/,teams/,0,0
